In [2]:
# FINAL FUNCTION + STANCE AND SWING PHASES WITH SMOOTHING AND VISUALIZATION
import pandas as pd
import numpy as np
import os
from scipy.signal import find_peaks
from scipy.ndimage import gaussian_filter1d
import matplotlib.pyplot as plt

# Directory paths
input_folder = r"C:\Users\diyav\.jupyter\Data"  # Specify your input folder path
output_folder = r"C:\Users\diyav\.jupyter\Filtered Data"  # Specify your output folder path

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Function to segment data into stance and swing phases
def segment_stance_and_swing(data, peaks, valleys):
    """Segments data into stance and swing phases."""
    stance_phases = []
    swing_phases = []

    for i in range(len(valleys) - 1):
        # Stance phase: From the last heel strike (peak) to the current toe-off (valley)
        prev_peak = peaks[peaks < valleys[i]].max() if any(peaks < valleys[i]) else None
        if prev_peak is not None and valleys[i] > prev_peak:
            stance_phases.append(data.iloc[prev_peak:valleys[i]])

        # Swing phase: From the current toe-off (valley) to the next heel strike (peak)
        next_peak = peaks[peaks > valleys[i]].min() if any(peaks > valleys[i]) else None
        if next_peak is not None and next_peak > valleys[i]:
            swing_phases.append(data.iloc[valleys[i]:next_peak])

    return stance_phases, swing_phases

# Function for peak detection with smoothing
def detect_peaks_and_valleys(z_axis_data, sigma=0.5, distance=10, threshold_percentile=98):
    """Detects peaks (heel strikes) and valleys (toe-offs) with smoothing and minimum distance."""
    # Smooth the data using a Gaussian filter
    z_axis_smoothed = gaussian_filter1d(z_axis_data, sigma=sigma)

    # Detect peaks and valleys with a minimum distance constraint
    peaks, _ = find_peaks(z_axis_smoothed, distance=distance)
    valleys, _ = find_peaks(-z_axis_smoothed, distance=distance)

    # Calculate thresholds for abnormal peaks and valleys
    peak_differences = np.diff(z_axis_smoothed[peaks])
    valley_differences = np.diff(z_axis_smoothed[valleys])
    threshold = np.percentile(np.abs(peak_differences), threshold_percentile)
    valley_threshold = np.percentile(np.abs(valley_differences), threshold_percentile)

    # Filter out abnormal peaks and valleys
    abnormal_peaks = np.where(np.abs(peak_differences) > threshold)[0] + 1
    abnormal_valleys = np.where(np.abs(valley_differences) > valley_threshold)[0] + 1
    peaks_filtered = np.delete(peaks, abnormal_peaks)
    valleys_filtered = np.delete(valleys, abnormal_valleys)

    return z_axis_smoothed, peaks_filtered, valleys_filtered

In [3]:
# Iterate through each CSV file in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.csv'):  # Check if the file is a CSV
        input_file_path = os.path.join(input_folder, filename)

        # Load the CSV data
        data = pd.read_csv(input_file_path)

        # Extract the Z-axis data for peak detection
        z_axis_data = data['Z_Accel'].values  # Adjust column name if needed

        # Detect peaks and valleys with smoothing and filtering
        z_axis_smoothed, peaks_filtered, valleys_filtered = detect_peaks_and_valleys(z_axis_data)

        # Segment the data into stance and swing phases
        stance_phases, swing_phases = segment_stance_and_swing(data, peaks_filtered, valleys_filtered)

        # Create a folder for each runthrough (CSV file) in the output directory
        runthrough_folder = os.path.join(output_folder, filename.replace('.csv', ''))
        os.makedirs(runthrough_folder, exist_ok=True)

        # Save each stance phase and swing phase as separate CSV files within the runthrough folder
        for i, stance in enumerate(stance_phases):
            stance_filename = f"stance_phase_{i + 1}.csv"
            stance_file_path = os.path.join(runthrough_folder, stance_filename)
            stance.to_csv(stance_file_path, index=False)
            print(f"Saved: {stance_file_path}")

        for i, swing in enumerate(swing_phases):
            swing_filename = f"swing_phase_{i + 1}.csv"
            swing_file_path = os.path.join(runthrough_folder, swing_filename)
            swing.to_csv(swing_file_path, index=False)
            print(f"Saved: {swing_file_path}")

        # Visualization
        plt.figure(figsize=(12, 6))
        plt.plot(z_axis_data, label='Raw Data', alpha=0.5)
        plt.plot(z_axis_smoothed, label='Smoothed Data', linewidth=2)
        plt.scatter(peaks_filtered, z_axis_smoothed[peaks_filtered], color='green', label='Peaks (Heel Strikes)', zorder=5)
        plt.scatter(valleys_filtered, z_axis_smoothed[valleys_filtered], color='red', label='Valleys (Toe-Offs)', zorder=5)
        plt.title(f"Peak and Valley Detection for {filename}")
        plt.xlabel('Time (Samples)')
        plt.ylabel('Z-Axis Acceleration')
        plt.legend()
        plt.grid(True)

        # Save the visualization plot
        plot_filename = os.path.join(runthrough_folder, f"{filename.replace('.csv', '')}_peaks_valleys_plot.png")
        plt.savefig(plot_filename)
        plt.close()
        print(f"Saved visualization plot: {plot_filename}")

print(f"All stance and swing phases have been segmented and saved to: {output_folder}")

Saved: C:\Users\diyav\.jupyter\Filtered Data\sub10_2_normal\stance_phase_1.csv
Saved: C:\Users\diyav\.jupyter\Filtered Data\sub10_2_normal\stance_phase_2.csv
Saved: C:\Users\diyav\.jupyter\Filtered Data\sub10_2_normal\stance_phase_3.csv
Saved: C:\Users\diyav\.jupyter\Filtered Data\sub10_2_normal\stance_phase_4.csv
Saved: C:\Users\diyav\.jupyter\Filtered Data\sub10_2_normal\stance_phase_5.csv
Saved: C:\Users\diyav\.jupyter\Filtered Data\sub10_2_normal\stance_phase_6.csv
Saved: C:\Users\diyav\.jupyter\Filtered Data\sub10_2_normal\stance_phase_7.csv
Saved: C:\Users\diyav\.jupyter\Filtered Data\sub10_2_normal\stance_phase_8.csv
Saved: C:\Users\diyav\.jupyter\Filtered Data\sub10_2_normal\stance_phase_9.csv
Saved: C:\Users\diyav\.jupyter\Filtered Data\sub10_2_normal\stance_phase_10.csv
Saved: C:\Users\diyav\.jupyter\Filtered Data\sub10_2_normal\stance_phase_11.csv
Saved: C:\Users\diyav\.jupyter\Filtered Data\sub10_2_normal\stance_phase_12.csv
Saved: C:\Users\diyav\.jupyter\Filtered Data\sub1

In [4]:
# FINAL FUNCTION + STANCE AND SWING PHASES WITH SMOOTHING, SEGMENTATION, AND SAVING
import pandas as pd
import numpy as np
import os
from scipy.signal import find_peaks
from scipy.ndimage import gaussian_filter1d
import matplotlib.pyplot as plt

# Directory paths
input_folder = r"C:\Users\diyav\.jupyter\Data"  # Specify your input folder path
output_folder = r"C:\Users\diyav\.jupyter\Filtered Data"  # Specify your output folder path

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Function for peak detection with smoothing
def detect_peaks_and_valleys(z_axis_data, sigma=0.5, distance=10, threshold_percentile=98):
    """Detects peaks (heel strikes) and valleys (toe-offs) with smoothing and minimum distance."""
    # Smooth the data using a Gaussian filter
    z_axis_smoothed = gaussian_filter1d(z_axis_data, sigma=sigma)

    # Detect peaks and valleys with a minimum distance constraint
    peaks, _ = find_peaks(z_axis_smoothed, distance=distance)
    valleys, _ = find_peaks(-z_axis_smoothed, distance=distance)

    # Calculate thresholds for abnormal peaks and valleys
    peak_differences = np.diff(z_axis_smoothed[peaks])
    valley_differences = np.diff(z_axis_smoothed[valleys])
    threshold = np.percentile(np.abs(peak_differences), threshold_percentile)
    valley_threshold = np.percentile(np.abs(valley_differences), threshold_percentile)

    # Filter out abnormal peaks and valleys
    abnormal_peaks = np.where(np.abs(peak_differences) > threshold)[0] + 1
    abnormal_valleys = np.where(np.abs(valley_differences) > valley_threshold)[0] + 1
    peaks_filtered = np.delete(peaks, abnormal_peaks)
    valleys_filtered = np.delete(valleys, abnormal_valleys)

    return z_axis_smoothed, peaks_filtered, valleys_filtered

# Function to segment data into stance and swing phases based on 60%-40% ratio
def segment_stance_and_swing(data, peaks, valleys):
    """Segments data into stance and swing phases based on the ratio of 60% stance and 40% swing."""
    stance_phases = []
    swing_phases = []

    for i in range(len(valleys) - 1):
        # Define gait cycle boundaries
        gait_cycle_start = valleys[i]
        gait_cycle_end = valleys[i + 1]
        gait_cycle_duration = gait_cycle_end - gait_cycle_start

        # Calculate indices for stance and swing phases
        stance_end = gait_cycle_start + int(0.6 * gait_cycle_duration)
        swing_start = stance_end

        # Append stance and swing phases
        stance_phases.append(data.iloc[gait_cycle_start:stance_end])
        swing_phases.append(data.iloc[swing_start:gait_cycle_end])

    return stance_phases, swing_phases

# Iterate through each CSV file in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.csv'):  # Check if the file is a CSV
        input_file_path = os.path.join(input_folder, filename)

        # Load the CSV data
        data = pd.read_csv(input_file_path)

        # Extract the Z-axis data for peak detection
        z_axis_data = data['Z_Accel'].values  # Adjust column name if needed

        # Detect peaks and valleys with smoothing and filtering
        z_axis_smoothed, peaks_filtered, valleys_filtered = detect_peaks_and_valleys(z_axis_data)

        # Segment the data into stance and swing phases
        stance_phases, swing_phases = segment_stance_and_swing(data, peaks_filtered, valleys_filtered)

        # Create a folder for each runthrough (CSV file) in the output directory
        runthrough_folder = os.path.join(output_folder, filename.replace('.csv', ''))
        os.makedirs(runthrough_folder, exist_ok=True)

        # Save each stance phase and swing phase as separate CSV files within the runthrough folder
        for i, stance in enumerate(stance_phases):
            stance_filename = f"stance_phase_{i + 1}.csv"
            stance_file_path = os.path.join(runthrough_folder, stance_filename)
            stance.to_csv(stance_file_path, index=False)
            print(f"Saved: {stance_file_path}")

        for i, swing in enumerate(swing_phases):
            swing_filename = f"swing_phase_{i + 1}.csv"
            swing_file_path = os.path.join(runthrough_folder, swing_filename)
            swing.to_csv(swing_file_path, index=False)
            print(f"Saved: {swing_file_path}")

        # Visualization
        plt.figure(figsize=(12, 6))
        plt.plot(z_axis_data, label='Raw Data', alpha=0.5)
        plt.plot(z_axis_smoothed, label='Smoothed Data', linewidth=2)
        plt.scatter(peaks_filtered, z_axis_smoothed[peaks_filtered], color='green', label='Peaks (Heel Strikes)', zorder=5)
        plt.scatter(valleys_filtered, z_axis_smoothed[valleys_filtered], color='red', label='Valleys (Toe-Offs)', zorder=5)
        plt.title(f"Peak and Valley Detection for {filename}")
        plt.xlabel('Time (Samples)')
        plt.ylabel('Z-Axis Acceleration')
        plt.legend()
        plt.grid(True)

        # Save the visualization plot
        plot_filename = os.path.join(runthrough_folder, f"{filename.replace('.csv', '')}_peaks_valleys_plot.png")
        plt.savefig(plot_filename)
        plt.close()
        print(f"Saved visualization plot: {plot_filename}")

print(f"All stance and swing phases have been segmented and saved to: {output_folder}")


Saved: C:\Users\diyav\.jupyter\Filtered Data\sub10_2_normal\stance_phase_1.csv
Saved: C:\Users\diyav\.jupyter\Filtered Data\sub10_2_normal\stance_phase_2.csv
Saved: C:\Users\diyav\.jupyter\Filtered Data\sub10_2_normal\stance_phase_3.csv
Saved: C:\Users\diyav\.jupyter\Filtered Data\sub10_2_normal\stance_phase_4.csv
Saved: C:\Users\diyav\.jupyter\Filtered Data\sub10_2_normal\stance_phase_5.csv
Saved: C:\Users\diyav\.jupyter\Filtered Data\sub10_2_normal\stance_phase_6.csv
Saved: C:\Users\diyav\.jupyter\Filtered Data\sub10_2_normal\stance_phase_7.csv
Saved: C:\Users\diyav\.jupyter\Filtered Data\sub10_2_normal\stance_phase_8.csv
Saved: C:\Users\diyav\.jupyter\Filtered Data\sub10_2_normal\stance_phase_9.csv
Saved: C:\Users\diyav\.jupyter\Filtered Data\sub10_2_normal\stance_phase_10.csv
Saved: C:\Users\diyav\.jupyter\Filtered Data\sub10_2_normal\stance_phase_11.csv
Saved: C:\Users\diyav\.jupyter\Filtered Data\sub10_2_normal\stance_phase_12.csv
Saved: C:\Users\diyav\.jupyter\Filtered Data\sub1